In [1]:
from sklearn import datasets, cross_validation
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from subprocess import check_output 

dataset=pd.read_csv('./intial_trainig_stage.csv',index_col='date',parse_dates=['date'])

ImportError: cannot import name 'cross_validation' from 'sklearn' (C:\ProgramData\Anaconda3\lib\site-packages\sklearn\__init__.py)

In [ ]:
datalen=len(dataset)
trainlen=int(datalen*0.8)
testlen=datalen-trainlen

In [ ]:
training_set=dataset.iloc[0:trainlen,0].values
test_set=dataset.iloc[trainlen:datalen,0].values

In [ ]:
#변경 이전
X_train=[]
y_train=[]
for i in range(0,trainlen-1):
    X_train.append(training_set[i])
    y_train.append(training_set[i+1])

In [ ]:
#변경 이후
training_set_pair=[]
for i in range(0,trainlen-1):
    training_set_pair.append([training_set[i],training_set[i+1]])

In [ ]:
X_train=np.array(X_train)
X_train

anomaly detection - 일단 training set에서만 anomaly detection 탐지 및 삭제함

In [ ]:
import rrcf

# Set tree parameters
num_trees = 40
shingle_size = 10
tree_size = 256

# Create a forest of empty trees
forest = []
for _ in range(num_trees):
    tree = rrcf.RCTree()            
    forest.append(tree)
    
# Use the "shingle" generator to create rolling window
points = rrcf.shingle(X_train, size=shingle_size)

# Create a dict to store anomaly score of each point
avg_codisp = {}

In [ ]:
point_len=len(X_train)
point_len

In [ ]:
# For each shingle...
for index, point in enumerate(points): #enumerate(): (인덱스 번호, 컬렉션의 원소) tuple을 반환
    # For each tree in the forest...
    for tree in forest:
        # If tree is above permitted size, drop the oldest point (FIFO)
        if len(tree.leaves) > tree_size:
            tree.forget_point(index - tree_size)
        # Insert the new point into the tree
        tree.insert_point(point, index=index)
        # Compute codisp on the new point and take the average among all trees
        if not index in avg_codisp:  # 공분산 계수 계산
            avg_codisp[index] = 0
        avg_codisp[index] += tree.codisp(index) / num_trees 

In [ ]:
len(avg_codisp)

In [ ]:
avg_codisp

Z-score: 평균값에서 표준편차의 몇배 정도 떨어져 있다 - 개별 데이터 값에서 평균을 빼고 그 값을 표준편차로 나눈 값

In [ ]:
score_mean=np.array(list(avg_codisp.values())).mean()
score_mean

In [ ]:
score_std=np.array(list(avg_codisp.values())).std()
score_std

In [ ]:
new_training_set=[]

for i in range(0,len(avg_codisp)):
    t=(avg_codisp[i]-score_mean)/score_std
    if t<=3:
        new_training_set.append(training_set_pair[i])

In [ ]:
X_train=[]
y_train=[]
for i in range(0,len(new_training_set)):
    X_train.append(new_training_set[i][0])
    y_train.append(new_training_set[i][1])

In [ ]:
# 변경 이전

X_test=[]
y_test=[]
i=0
for i in range(0,testlen-1):
    X_test.append(test_set[i])
    y_test.append(test_set[i+1])

In [ ]:
# 주석
#list -> ndim array로 변환
X_train=np.reshape(X_train,(-1,1))
y_train=np.reshape(y_train,(-1,1))
X_test=np.reshape(X_test,(-1,1))
y_test=np.reshape(y_test,(-1,1))

In [ ]:
from skgarden import MondrianForestRegressor
mfr=MondrianForestRegressor(random_state=1,n_estimators=100,max_depth=50,bootstrap=True)
mfr.partial_fit(X_train,y_train)
y_pred,y_std=mfr.predict(X_test,return_std=True)

In [ ]:
plt.plot(y_test,color='red',label='Real Price')
plt.plot(y_pred,color='blue',label='Predicted Price')
plt.xlabel('Time')
plt.ylabel('Stock Price')

plt.legend()
plt.show()

In [ ]:
mfr.score(y_test,y_pred)  # regression을 위한 accuracy_score